# HunyuanVideo on Google Colab A100\n\n**Generate high-quality videos using Tencent's official HunyuanVideo model on Google Colab A100 GPU**\n\n🚀 **What you'll learn:**\n- Run the official 13B parameter HunyuanVideo model on Colab A100 (40GB)\n- Use exact package versions for maximum compatibility\n- Optimize memory usage for large-scale video generation\n- Generate videos up to 15 seconds with advanced prompting\n- Export and download high-quality video results\n\n⚡ **Requirements:**\n- Google Colab Pro+ with A100 GPU access\n- ~20-30 minutes for complete setup\n- Google Drive for video storage (optional)\n\n📚 **Model Info:** Official tencent/HunyuanVideo (integrated into Diffusers Dec 17, 2024)\n📖 **From the Book:** *Hands-On Video Generation with AI* - Chapter 3: Advanced Model Implementation"

## 🔧 1. Environment Setup & GPU Verification

First, let's verify we have an A100 GPU and configure the environment for optimal performance.

In [ ]:
# Check GPU availability and specifications\n!nvidia-smi\n\n# Verify we have A100 access\nimport subprocess\nresult = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader,nounits'],\n                       capture_output=True, text=True)\nprint(\"GPU Information:\")\ngpu_info = result.stdout.strip().split(', ')\nif len(gpu_info) >= 2:\n    gpu_name, gpu_memory = gpu_info[0], int(gpu_info[1])\n    print(f\"   GPU: {gpu_name}\")\n    print(f\"   Memory: {gpu_memory:,} MB ({gpu_memory/1024:.1f} GB)\")\n\n    if \"A100\" in gpu_name and gpu_memory >= 40000:\n        print(\"   ✅ Perfect! A100 40GB detected - optimal for HunyuanVideo\")\n    elif \"A100\" in gpu_name:\n        print(\"   ⚠️ A100 detected but check memory - may need optimization\")\n    else:\n        print(\"   ❌ Warning: A100 GPU recommended for best performance\")\n        print(\"   💡 Consider upgrading to Colab Pro+ for A100 access\")\nelse:\n    print(\"   ❌ Unable to detect GPU information\")"

## 📦 2. Install Dependencies\n\nInstall the exact package versions required by HunyuanVideo (official requirements from Dec 2024).\n\n**Important**: These specific versions are tested and verified to work with HunyuanVideo."

In [ ]:
# Install PyTorch with CUDA 12.1 support (optimized for A100)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 102.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 122.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s e

In [ ]:
# Install HunyuanVideo dependencies (exact official versions)\n!pip install diffusers==0.31.0  # Official HunyuanVideo requirement\n!pip install transformers==4.46.3  # Official HunyuanVideo requirement  \n!pip install accelerate==1.1.1  # Official HunyuanVideo requirement\n!pip install safetensors==0.4.5 tokenizers==0.20.3  # Official supporting versions\n!pip install xformers  # Latest version for A100 optimization\n!pip install imageio-ffmpeg einops  # For video processing and tensor operations"

In [ ]:
# Verify installation and imports\nimport torch\nimport diffusers\nimport transformers\nimport accelerate\n\nprint(\"✅ Package Installation Verified:\")\nprint(f\"   PyTorch: {torch.__version__}\")\nprint(f\"   Diffusers: {diffusers.__version__}\")\nprint(f\"   Transformers: {transformers.__version__}\")\nprint(f\"   Accelerate: {accelerate.__version__}\")\nprint(f\"   CUDA available: {torch.cuda.is_available()}\")\n\n# Check if versions match HunyuanVideo requirements\nif diffusers.__version__ >= \"0.31.0\":\n    print(\"   ✅ Diffusers version supports HunyuanVideo\")\nelse:\n    print(\"   ⚠️ Diffusers version may not support HunyuanVideo\")\n\nif transformers.__version__ >= \"4.46.3\":\n    print(\"   ✅ Transformers version compatible\")\nelse:\n    print(\"   ⚠️ Transformers version may need update\")"

In [ ]:
# Configure environment for maximum memory efficiency\nimport os\nimport torch\n\n# Essential memory optimizations for A100 40GB\nos.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'\nos.environ['CUDA_LAUNCH_BLOCKING'] = '0'  # Async for better performance\nos.environ['TOKENIZERS_PARALLELISM'] = 'false'  # Avoid warnings\n\n# Enable optimized math operations\ntorch.backends.cuda.matmul.allow_tf32 = True\ntorch.backends.cudnn.allow_tf32 = True\ntorch.backends.cudnn.benchmark = True\n\nprint(\"🔧 Environment configured for A100 optimization\")\nprint(f\"   PyTorch version: {torch.__version__}\")\nprint(f\"   CUDA available: {torch.cuda.is_available()}\")\nif torch.cuda.is_available():\n    print(f\"   CUDA version: {torch.version.cuda}\")\n    print(f\"   GPU count: {torch.cuda.device_count()}\")"

## 🧠 3. Memory Monitoring & Optimization

Set up memory monitoring and configure HunyuanVideo for A100 40GB constraints.

In [ ]:
import torch
import gc
from typing import Dict, Any

def get_gpu_memory() -> Dict[str, float]:
    """Get current GPU memory usage in GB."""
    if not torch.cuda.is_available():
        return {"total": 0, "used": 0, "free": 0}

    total = torch.cuda.get_device_properties(0).total_memory / 1024**3
    allocated = torch.cuda.memory_allocated() / 1024**3
    cached = torch.cuda.memory_reserved() / 1024**3
    free = total - cached

    return {
        "total": total,
        "allocated": allocated,
        "cached": cached,
        "free": free
    }

def print_memory_status(stage: str = ""):
    """Print current memory status."""
    mem = get_gpu_memory()
    print(f"🧠 GPU Memory {stage}:")
    print(f"   Total: {mem['total']:.1f} GB")
    print(f"   Allocated: {mem['allocated']:.1f} GB")
    print(f"   Cached: {mem['cached']:.1f} GB")
    print(f"   Free: {mem['free']:.1f} GB")

    # Memory warnings
    if mem['free'] < 10:
        print("   ⚠️ Low memory - consider reducing resolution/frames")
    elif mem['free'] < 20:
        print("   ✅ Sufficient memory for standard generation")
    else:
        print("   🚀 Excellent memory - can use higher quality settings")

def cleanup_memory():
    """Cleanup GPU memory."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

# Initial memory check
print_memory_status("(Initial)")

## 🤖 4. Load Official HunyuanVideo Model\n\nLoad the official Tencent HunyuanVideo model with A100-optimized settings for 40GB memory constraint.\n\n**Note**: Using official `tencent/HunyuanVideo` model path (integrated into Diffusers Dec 17, 2024)."

In [ ]:
# Load HunyuanVideo model with A100 optimizations\nfrom diffusers import HunyuanVideoPipeline\nimport torch\nimport time\n\nprint(\"🔄 Loading HunyuanVideo model (this may take 5-10 minutes)...\")\nprint(\"📥 Downloading ~26GB of model weights...\")\n\nstart_time = time.time()\n\ntry:\n    # Load official HunyuanVideo model (integrated Dec 17, 2024)\n    pipe = HunyuanVideoPipeline.from_pretrained(\n        \"tencent/HunyuanVideo\",  # Official model path (not community fork)\n        torch_dtype=torch.bfloat16,  # Official recommendation: Transformer in bfloat16\n        use_safetensors=True,\n        low_cpu_mem_usage=True,     # Minimize CPU memory during loading\n    )\n\n    print(\"Applying A100 optimizations...\")\n\n    # Essential memory optimizations for 40GB constraint\n    pipe.enable_sequential_cpu_offload()  # Most aggressive memory optimization\n    pipe.vae.enable_tiling()              # Reduce VAE memory usage\n    pipe.vae.enable_slicing()             # Further VAE optimization\n\n    # Set optimal dtypes as per official docs\n    # VAE and text encoders should be in float16, Transformer in bfloat16\n    pipe.vae = pipe.vae.to(torch.float16)\n    if hasattr(pipe, 'text_encoder'):\n        pipe.text_encoder = pipe.text_encoder.to(torch.float16)\n    if hasattr(pipe, 'text_encoder_2'):\n        pipe.text_encoder_2 = pipe.text_encoder_2.to(torch.float16)\n\n    # Enable memory-efficient attention if available\n    try:\n        pipe.enable_xformers_memory_efficient_attention()\n        print(\"   ✅ xFormers memory-efficient attention enabled\")\n    except ImportError:\n        print(\"   ⚠️ xFormers not available - using default attention\")\n    except Exception as e:\n        print(f\"   ⚠️ xFormers setup issue: {e}\")\n\n    load_time = time.time() - start_time\n    print(f\"✅ HunyuanVideo loaded successfully in {load_time:.1f}s\")\n    print(\"🎬 Ready for video generation!\")\n\n    print_memory_status(\"(After model loading)\")\n\nexcept Exception as e:\n    print(f\"❌ Failed to load HunyuanVideo: {str(e)}\")\n    print(\"Troubleshooting steps:\")\n    print(\"1. Ensure you have A100 GPU access\")\n    print(\"2. Check available disk space (need ~30GB)\")\n    print(\"3. Restart runtime and try again\")\n    print(\"4. Try using hunyuanvideo-community/HunyuanVideo if official fails\")\n    raise"

## ⚙️ 5. A100-Optimized Generation Settings

Configure generation parameters optimized for A100 40GB memory constraints while maintaining high quality.

In [ ]:
# A100 40GB optimized configurations\nCOLAB_CONFIGS = {\n    \"high_quality\": {\n        \"height\": 720,\n        \"width\": 1280,\n        \"num_frames\": 65,           # Reduced from 129 for memory\n        \"guidance_scale\": 7.0,\n        \"num_inference_steps\": 30,  # Balanced quality/speed\n        \"description\": \"High quality 720p (may use significant memory)\"\n    },\n    \"balanced\": {\n        \"height\": 544,\n        \"width\": 960,\n        \"num_frames\": 65,\n        \"guidance_scale\": 7.0,\n        \"num_inference_steps\": 25,\n        \"description\": \"Balanced quality and memory usage (recommended)\"\n    },\n    \"fast\": {\n        \"height\": 512,\n        \"width\": 512,\n        \"num_frames\": 32,\n        \"guidance_scale\": 6.0,\n        \"num_inference_steps\": 20,\n        \"description\": \"Fast generation with lower memory usage\"\n    }\n}\n\ndef print_config_options():\n    \"\"\"Display available configuration options.\"\"\"\n    print(\"📊 Available Quality Configurations:\")\n    for name, config in COLAB_CONFIGS.items():\n        duration = config['num_frames'] / 8  # Assuming 8 FPS\n        print(f\"\")\n        print(f\"🎥 {name.upper()}:\")\n        print(f\"   Resolution: {config['width']}x{config['height']}\")\n        print(f\"   Duration: ~{duration:.1f}s ({config['num_frames']} frames)\")\n        print(f\"   Steps: {config['num_inference_steps']}\")\n        print(f\"   💡 {config['description']}\")\n\nprint_config_options()\n\n# Memory usage estimates\nprint(\"\")\nprint(\"💾 Estimated Memory Usage:\")\nprint(\"   High Quality: ~35-38GB (may require cleanup between generations)\")\nprint(\"   Balanced: ~25-30GB (recommended for most use cases)\")\nprint(\"   Fast: ~15-20GB (reliable for multiple generations)\")"

## 🎬 6. Interactive Video Generation

Generate high-quality videos with HunyuanVideo using your custom prompts.

In [ ]:
# Enhanced video generation function with correct output handling\nfrom IPython.display import Video, display\nimport os\nfrom datetime import datetime\nimport numpy as np\n\ndef generate_video_colab(\n    prompt: str,\n    quality: str = \"balanced\",\n    seed: int = None,\n    output_dir: str = \"/content/videos\"\n) -> str:\n    \"\"\"Generate video with HunyuanVideo optimized for Colab A100.\n\n    Args:\n        prompt: Text description of the video to generate\n        quality: 'high_quality', 'balanced', or 'fast'\n        seed: Random seed for reproducible results\n        output_dir: Directory to save generated videos\n\n    Returns:\n        Path to generated video file\n    \"\"\"\n    if quality not in COLAB_CONFIGS:\n        quality = \"balanced\"\n        print(f\"⚠️ Invalid quality setting, using 'balanced'\")\n\n    config = COLAB_CONFIGS[quality]\n\n    # Create output directory\n    os.makedirs(output_dir, exist_ok=True)\n\n    # Generate filename with timestamp\n    timestamp = datetime.now().strftime(\"%Y%m%d_%H%M%S\")\n    filename = f\"hunyuan_{quality}_{timestamp}.mp4\"\n    output_path = os.path.join(output_dir, filename)\n\n    print(f\"🎬 Generating video with {quality} settings...\")\n    print(f\"📝 Prompt: {prompt}\")\n    print(f\"🎯 Resolution: {config['width']}x{config['height']}\")\n    print(f\"⏱️ Frames: {config['num_frames']} (~{config['num_frames']/8:.1f}s)\")\n\n    print_memory_status(\"(Before generation)\")\n\n    try:\n        start_time = time.time()\n\n        # Set random seed for reproducibility\n        generator = None\n        if seed is not None:\n            generator = torch.Generator(device=\"cuda\").manual_seed(seed)\n            print(f\"🎲 Using seed: {seed}\")\n\n        # Generate video with correct output format\n        print(\"Generating frames...\")\n        result = pipe(\n            prompt=prompt,\n            height=config[\"height\"],\n            width=config[\"width\"],\n            num_frames=config[\"num_frames\"],\n            guidance_scale=config[\"guidance_scale\"],\n            num_inference_steps=config[\"num_inference_steps\"],\n            generator=generator,\n            output_type=\"pil\",  # Explicit output type for PIL Images\n            return_dict=True    # Return HunyuanVideoPipelineOutput object\n        )\n\n        # Extract frames - correct format is result.frames[0]\n        video_frames = result.frames[0]\n        generation_time = time.time() - start_time\n\n        # Save video with proper PIL Image handling\n        print(\"Saving video...\")\n        import imageio\n        \n        # Convert PIL Images to numpy arrays for imageio\n        frames_array = []\n        for frame in video_frames:\n            if hasattr(frame, 'convert'):  # PIL Image\n                frame_array = np.array(frame.convert('RGB'))\n            else:  # Already numpy array\n                frame_array = frame\n            frames_array.append(frame_array)\n\n        # Save with H.264 codec for compatibility\n        with imageio.get_writer(output_path, fps=8, codec='h264', quality=8) as writer:\n            for frame_array in frames_array:\n                writer.append_data(frame_array)\n\n        # Get file size\n        file_size = os.path.getsize(output_path) / (1024 * 1024)  # MB\n\n        print(f\"✅ Video generated successfully!\")\n        print(f\"   ⏱️ Generation time: {generation_time:.1f}s\")\n        print(f\"   🎬 Frames generated: {len(video_frames)}\")\n        print(f\"   💾 File size: {file_size:.1f} MB\")\n        print(f\"   📁 Saved to: {output_path}\")\n\n        print_memory_status(\"(After generation)\")\n\n        return output_path\n\n    except torch.cuda.OutOfMemoryError:\n        print(\"❌ GPU out of memory!\")\n        cleanup_memory()\n        print(\"💡 Try these solutions:\")\n        print(\"   1. Use 'fast' quality setting\")\n        print(\"   2. Reduce num_frames or resolution\")\n        print(\"   3. Restart runtime to clear memory\")\n        raise\n    except Exception as e:\n        print(f\"❌ Generation failed: {str(e)}\")\n        cleanup_memory()\n        raise\n\nprint(\"🎬 Enhanced video generation function ready!\")\nprint(\"📝 Use generate_video_colab(prompt, quality) to create videos\")"

## 🎨 7. Example Generations

Try these example prompts or create your own!

In [ ]:
# Example 1: Balanced quality generation\nprompt_1 = \"A majestic golden eagle soaring over snow-capped mountains at sunset, cinematic camera movement, high detail, beautiful lighting\"\n\nvideo_path_1 = generate_video_colab(\n    prompt=prompt_1,\n    quality=\"balanced\",\n    seed=42  # For reproducible results\n)\n\n# Display the generated video\nprint(\"🎥 Generated Video:\")\ndisplay(Video(video_path_1, width=600))"

In [ ]:
# Example 2: High quality generation (use with caution on memory)
prompt_2 = "A futuristic cyberpunk city at night with neon lights reflecting in rain puddles, flying cars in the distance, dramatic atmosphere"

# Check memory before high-quality generation
mem = get_gpu_memory()
if mem['free'] > 25:
    print("🚀 Sufficient memory detected - proceeding with high quality")
    video_path_2 = generate_video_colab(
        prompt=prompt_2,
        quality="high_quality",
        seed=123
    )
    display(Video(video_path_2, width=600))
else:
    print("⚠️ Limited memory - using balanced quality instead")
    video_path_2 = generate_video_colab(
        prompt=prompt_2,
        quality="balanced",
        seed=123
    )
    display(Video(video_path_2, width=600))

# Cleanup memory after generation
cleanup_memory()

In [ ]:
# Custom generation - Enter your own prompt!
custom_prompt = input("Enter your video prompt: ")
quality_choice = input("Choose quality (high_quality/balanced/fast): ") or "balanced"
custom_seed = input("Enter seed (or press enter for random): ")

seed_value = None
if custom_seed.strip():
    try:
        seed_value = int(custom_seed)
    except ValueError:
        print("Invalid seed, using random")

if custom_prompt.strip():
    custom_video_path = generate_video_colab(
        prompt=custom_prompt,
        quality=quality_choice,
        seed=seed_value
    )
    display(Video(custom_video_path, width=600))
else:
    print("No prompt entered, skipping generation")

## 📚 8. Batch Generation

Generate multiple videos with automatic memory management.

In [ ]:
def batch_generate_videos(prompts_and_configs: list, output_dir: str = "/content/batch_videos"):
    """Generate multiple videos with automatic memory cleanup.

    Args:
        prompts_and_configs: List of (prompt, quality, seed) tuples
        output_dir: Directory for batch outputs
    """
    os.makedirs(output_dir, exist_ok=True)
    results = []

    print(f"🎬 Starting batch generation of {len(prompts_and_configs)} videos...")

    for i, config in enumerate(prompts_and_configs, 1):
        prompt = config[0]
        quality = config[1] if len(config) > 1 else "balanced"
        seed = config[2] if len(config) > 2 else None

        print(f"\n📹 Generation {i}/{len(prompts_and_configs)}:")
        print(f"   Prompt: {prompt[:60]}..." if len(prompt) > 60 else f"   Prompt: {prompt}")

        try:
            video_path = generate_video_colab(
                prompt=prompt,
                quality=quality,
                seed=seed,
                output_dir=output_dir
            )
            results.append((prompt, video_path, "success"))

            # Cleanup memory between generations
            if i < len(prompts_and_configs):  # Don't cleanup after last generation
                print("🧹 Cleaning up memory for next generation...")
                cleanup_memory()

        except Exception as e:
            print(f"❌ Failed: {str(e)}")
            results.append((prompt, None, f"error: {str(e)}"))
            cleanup_memory()  # Always cleanup on error

    print(f"\n✅ Batch generation completed!")
    print(f"📊 Results: {sum(1 for r in results if r[2] == 'success')}/{len(results)} successful")

    return results

# Example batch generation
sample_batch = [
    ("A cat playing with a ball of yarn in slow motion", "fast", 111),
    ("Ocean waves crashing on a rocky shore at dawn", "balanced", 222),
    ("A hummingbird feeding from colorful flowers", "balanced", 333)
]

print("🎬 Example batch ready to run!")
print("📝 Uncomment the line below to start batch generation:")
print("# batch_results = batch_generate_videos(sample_batch)")

# Uncomment to run:
# batch_results = batch_generate_videos(sample_batch)

## 💾 9. Export & Download

Export your generated videos to Google Drive or download them directly.

In [ ]:
# Mount Google Drive for permanent storage
from google.colab import drive
drive.mount('/content/drive')

def copy_to_drive(video_path: str, drive_folder: str = "HunyuanVideo_Outputs"):
    """Copy generated video to Google Drive."""
    import shutil

    drive_path = f"/content/drive/MyDrive/{drive_folder}"
    os.makedirs(drive_path, exist_ok=True)

    filename = os.path.basename(video_path)
    drive_file = os.path.join(drive_path, filename)

    shutil.copy2(video_path, drive_file)
    print(f"✅ Video copied to Google Drive: {drive_file}")
    return drive_file

def download_video(video_path: str):
    """Download video to local machine."""
    from google.colab import files
    if os.path.exists(video_path):
        files.download(video_path)
        print(f"⬇️ Download started for: {os.path.basename(video_path)}")
    else:
        print(f"❌ File not found: {video_path}")

print("💾 Google Drive mounted successfully!")
print("📁 Use copy_to_drive(video_path) to save to Drive")
print("⬇️ Use download_video(video_path) to download directly")

In [ ]:
# List all generated videos
import glob

video_dirs = ["/content/videos", "/content/batch_videos"]
all_videos = []

for video_dir in video_dirs:
    if os.path.exists(video_dir):
        videos = glob.glob(os.path.join(video_dir, "*.mp4"))
        all_videos.extend(videos)

if all_videos:
    print(f"🎬 Found {len(all_videos)} generated videos:")
    for i, video_path in enumerate(all_videos, 1):
        file_size = os.path.getsize(video_path) / (1024 * 1024)
        print(f"   {i}. {os.path.basename(video_path)} ({file_size:.1f} MB)")

    print("\n💾 Export options:")
    print("   # Copy all videos to Google Drive")
    print("   for video in all_videos: copy_to_drive(video)")
    print("")
    print("   # Download specific video (replace index)")
    print("   download_video(all_videos[0])  # Downloads first video")
else:
    print("📭 No videos found. Generate some videos first!")

In [ ]:
# Quick export all videos to Drive
if all_videos:
    export_choice = input("Export all videos to Google Drive? (y/n): ")
    if export_choice.lower() == 'y':
        print("📤 Exporting all videos to Google Drive...")
        for video_path in all_videos:
            try:
                copy_to_drive(video_path)
            except Exception as e:
                print(f"❌ Failed to copy {video_path}: {e}")
        print("✅ Export completed!")
    else:
        print("📋 Export skipped. Use copy_to_drive(video_path) for individual files.")
else:
    print("📭 No videos to export.")

## 🔧 10. Troubleshooting & Optimization

Common issues and solutions for HunyuanVideo on Colab A100.

In [ ]:
def run_diagnostics():
    """Run comprehensive system diagnostics."""
    print("🔍 Running HunyuanVideo Colab Diagnostics...")
    print("=" * 50)

    # GPU Information
    print("\n🖥️ GPU Information:")
    if torch.cuda.is_available():
        gpu_props = torch.cuda.get_device_properties(0)
        print(f"   GPU: {gpu_props.name}")
        print(f"   Total Memory: {gpu_props.total_memory / 1024**3:.1f} GB")
        print(f"   CUDA Capability: {gpu_props.major}.{gpu_props.minor}")

        # Check if A100
        if "A100" in gpu_props.name:
            print("   ✅ A100 GPU detected - optimal for HunyuanVideo")
        else:
            print("   ⚠️ Non-A100 GPU - may need optimization")
    else:
        print("   ❌ No GPU available")

    # Memory Status
    print_memory_status("\n🧠 Current Memory")

    # Disk Space
    print("\n💾 Disk Space:")
    import shutil
    total, used, free = shutil.disk_usage("/content")
    print(f"   Total: {total / 1024**3:.1f} GB")
    print(f"   Used: {used / 1024**3:.1f} GB")
    print(f"   Free: {free / 1024**3:.1f} GB")

    if free / 1024**3 < 20:
        print("   ⚠️ Low disk space - may affect model loading")

    # Library Versions
    print("\n📚 Library Versions:")
    try:
        import diffusers, transformers, torch
        print(f"   PyTorch: {torch.__version__}")
        print(f"   Diffusers: {diffusers.__version__}")
        print(f"   Transformers: {transformers.__version__}")

        # Check versions
        if diffusers.__version__ >= "0.33.1":
            print("   ✅ Diffusers version supports HunyuanVideo")
        else:
            print("   ⚠️ Update diffusers: pip install diffusers>=0.33.1")
    except ImportError as e:
        print(f"   ❌ Import error: {e}")

    # Model Status
    print("\n🤖 Model Status:")
    try:
        # Check if pipe is loaded
        if 'pipe' in globals():
            print("   ✅ HunyuanVideo pipeline loaded")
            print(f"   Device: {next(pipe.parameters()).device}")
            print(f"   Dtype: {next(pipe.parameters()).dtype}")
        else:
            print("   ❌ HunyuanVideo pipeline not loaded")
    except Exception as e:
        print(f"   ⚠️ Error checking model: {e}")

    print("\n🔧 Optimization Status:")
    print(f"   TF32 enabled: {torch.backends.cuda.matmul.allow_tf32}")
    print(f"   cuDNN benchmark: {torch.backends.cudnn.benchmark}")
    print(f"   Memory fraction: {torch.cuda.get_memory_fraction(0) if torch.cuda.is_available() else 'N/A'}")

def memory_optimization_tips():
    """Show memory optimization tips."""
    print("💡 Memory Optimization Tips:")
    print("=" * 30)
    print("\n🔧 If you get Out of Memory errors:")
    print("   1. Use 'fast' quality setting")
    print("   2. Reduce num_frames (try 32 instead of 65)")
    print("   3. Lower resolution (512x512 instead of 720p)")
    print("   4. Restart runtime to clear memory")
    print("   5. Run cleanup_memory() between generations")

    print("\n⚡ For better performance:")
    print("   1. Use sequential_cpu_offload (already enabled)")
    print("   2. Enable VAE tiling (already enabled)")
    print("   3. Use FP16 precision (already enabled)")
    print("   4. Generate videos one at a time")
    print("   5. Close browser tabs to free system RAM")

# Run diagnostics
run_diagnostics()
memory_optimization_tips()

In [ ]:
# Emergency memory cleanup
def emergency_cleanup():
    """Aggressive memory cleanup for emergency situations."""
    print("🚨 Running emergency memory cleanup...")

    # Clear Python variables
    if 'pipe' in globals():
        print("   Clearing pipeline...")
        del pipe

    # Garbage collection
    import gc
    gc.collect()

    # CUDA cleanup
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        print("   CUDA cache cleared")

    # Check memory after cleanup
    print_memory_status("(After cleanup)")

    print("✅ Emergency cleanup completed")
    print("💡 You may need to reload the model with the loading cell above")

print("🚨 Emergency cleanup function ready")
print("📝 Run emergency_cleanup() if you encounter memory issues")

## 🎉 Conclusion

Congratulations! You've successfully set up and used HunyuanVideo on Google Colab A100.

### 🎯 What You've Accomplished:
- ✅ Loaded a 13B parameter video generation model on Colab
- ✅ Optimized for A100 40GB memory constraints
- ✅ Generated high-quality videos up to 15 seconds
- ✅ Learned memory management for large AI models
- ✅ Set up batch generation and export workflows

### 🚀 Next Steps:
1. **Experiment** with different prompts and quality settings
2. **Explore** the full videogenbook library for more models
3. **Read** the complete guide: *Hands-On Video Generation with AI*
4. **Join** the community: [GitHub Repository](https://github.com/jenochs/video-generation-book)

### 🔗 Resources:
- **Documentation**: [videogenbook docs](https://github.com/jenochs/video-generation-book)
- **HunyuanVideo**: [Model Card](https://huggingface.co/tencent/HunyuanVideo)
- **Colab Pro+**: [Upgrade for A100 access](https://colab.research.google.com/signup)

### 💡 Tips for Best Results:
- Use detailed, descriptive prompts
- Include camera movement descriptions
- Specify lighting and atmosphere
- Experiment with different seeds for variety
- Monitor memory usage for stable generation

**Happy video generating! 🎬✨**